# Stochastic Volatility (Particle Filter) for Volatility Forecasting

Chapter 10 introduces stochastic volatility models with latent volatility.

We use a simple state-space model:

- Latent log-vol: $h_t = \mu + \phi (h_{t-1} - \mu) + \eta_t$, $\eta_t \sim \mathcal{N}(0, \sigma_\eta^2)$
- Returns: $r_t \sim \mathcal{N}(0, \exp(h_t))$

We run a bootstrap particle filter (SMC) with fixed parameters to estimate
filtered volatility. The forecasted volatility is used to create a
volatility-managed signal.


# Chapter 10 Bayesian ML (Predictive): Stochastic Volatility (Particle Filter)

These notebooks mirror the *methods* highlighted in
`ml_finance_thoery/machine-learning-for-trading/10_bayesian_machine_learning/README.md`
and apply them to the local `dataset/cleaned/` asset universe to produce
out-of-sample predictions and a backtest using the same **vectorized** engine
used by the `notebooks/ML_Linear_Models_*` notebooks.


In [1]:
from __future__ import annotations

import math
from pathlib import Path
import sys

import numpy as np
import pandas as pd

SEED = 42
rng = np.random.default_rng(SEED)

def find_project_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / 'src').exists() and (p / 'dataset').exists():
            return p
        p = p.parent
    raise RuntimeError(f'Could not find project root from: {start!s}')

PROJECT_ROOT = find_project_root(Path.cwd())
CLEANED_DIR = PROJECT_ROOT / 'dataset' / 'cleaned'

# Ensure `src/` is on sys.path so `backtester` is importable
src_dir = PROJECT_ROOT / 'src'
if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))


In [2]:
from backtester.data import load_cleaned_assets, align_close_prices

ASSET_LIMIT = 30
N_PARTICLES = 400

assets_ohlcv = load_cleaned_assets(cleaned_dir=str(CLEANED_DIR))
close_prices_full = align_close_prices(assets_ohlcv).sort_index()
assets = list(close_prices_full.columns)[:ASSET_LIMIT]
close_prices = close_prices_full.loc[:, assets]

rets = np.log(close_prices / close_prices.shift(1)).replace([np.inf, -np.inf], np.nan).dropna(how='all')
rets = rets.fillna(0.0)

# Time split
TRAIN_YEARS = 7
VAL_MONTHS = 18
TEST_MONTHS = 18

def align_to_trading_date(index: pd.DatetimeIndex, ts: pd.Timestamp) -> pd.Timestamp:
    pos = int(index.searchsorted(ts, side='left'))
    if pos >= len(index):
        return pd.Timestamp(index[-1])
    return pd.Timestamp(index[pos])

idx = pd.DatetimeIndex(rets.index).sort_values()
end = pd.Timestamp(idx[-1])
raw_test_start = end - pd.DateOffset(months=TEST_MONTHS)
raw_val_start = raw_test_start - pd.DateOffset(months=VAL_MONTHS)
raw_train_start = raw_val_start - pd.DateOffset(years=TRAIN_YEARS)
test_start = align_to_trading_date(idx, pd.Timestamp(raw_test_start))

# Fixed SV parameters (simple, robust defaults)
phi = 0.98
sigma_eta = 0.15

def particle_filter_sv(r: np.ndarray) -> np.ndarray:
    # Returns filtered vol (std) per time index
    r = np.asarray(r, dtype=float)
    # Initialize mu from empirical variance
    v = float(np.var(r))
    mu = float(np.log(v + 1e-12))
    h = rng.normal(loc=mu, scale=0.5, size=N_PARTICLES)
    w = np.full(N_PARTICLES, 1.0 / N_PARTICLES)
    vol = np.zeros(r.shape[0], dtype=float)
    for t in range(r.shape[0]):
        # propagate
        h = mu + phi * (h - mu) + rng.normal(0.0, sigma_eta, size=N_PARTICLES)
        # weight by likelihood N(0, exp(h))
        var = np.exp(h)
        ll = -0.5 * (np.log(2 * np.pi * var) + (r[t] ** 2) / var)
        ll = ll - np.max(ll)
        w = np.exp(ll)
        w = w / np.sum(w)
        # resample (systematic)
        cdf = np.cumsum(w)
        u0 = rng.random() / N_PARTICLES
        u = u0 + np.arange(N_PARTICLES) / N_PARTICLES
        idxs = np.searchsorted(cdf, u)
        h = h[idxs]
        w = np.full(N_PARTICLES, 1.0 / N_PARTICLES)
        vol[t] = float(np.sqrt(np.exp(np.mean(h))))
    return vol

# Compute filtered vol per asset (slow but manageable for ASSET_LIMIT)
vol_filt = pd.DataFrame(index=rets.index, columns=assets, dtype=float)
for a in assets:
    vol_filt[a] = particle_filter_sv(rets[a].to_numpy())

# Predictive signal: 5-day momentum scaled by next-day volatility forecast (use filtered vol as forecast)
mom_5d = rets.rolling(5).sum()
score = (mom_5d / (vol_filt + 1e-12)).loc[test_start:end]
pred_matrix = score.fillna(0.0)


In [3]:
from IPython.display import display
from bokeh.io import output_notebook, show

from backtester.data import load_cleaned_assets, align_close_prices
from backtester.engine import BacktestConfig, run_backtest
from backtester.report import compute_backtest_report
from backtester.bokeh_plots import build_interactive_portfolio_layout
from backtester.portfolio import equal_weight, optimize_mpt

output_notebook()

if 'pred_matrix' not in globals():
    raise RuntimeError('Expected `pred_matrix` (index=date, columns=Asset_ID) to exist')

# IMPORTANT: performance metrics should start when the model has signals.
# Keep the original prediction date range before reindexing to market data.
pred_range = pd.DatetimeIndex(pred_matrix.index).sort_values()
if pred_range.empty:
    raise RuntimeError('pred_matrix has empty index')
bt_start = pd.Timestamp(pred_range[0])
bt_end = pd.Timestamp(pred_range[-1])

bt_assets = sorted([str(c) for c in pred_matrix.columns.tolist()])
assets_ohlcv = load_cleaned_assets(symbols=bt_assets, cleaned_dir=str(CLEANED_DIR))
close_prices = align_close_prices(assets_ohlcv)

# Align prediction matrix to available trade dates
pred_matrix = pred_matrix.reindex(close_prices.index)
# Restrict backtest to the prediction window (avoid 2016-start metrics).
close_prices = close_prices.loc[bt_start:bt_end]
pred_matrix = pred_matrix.loc[bt_start:bt_end]
returns_matrix = close_prices.pct_change().fillna(0.0)

market_df = pd.DataFrame({
    'Open': pd.concat([df['Open'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'High': pd.concat([df['High'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Low': pd.concat([df['Low'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Close': pd.concat([df['Close'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Volume': pd.concat([df['Volume'] for df in assets_ohlcv.values()], axis=1).sum(axis=1),
}).sort_index().loc[bt_start:bt_end]

REBALANCE_FREQ = 'W'
TOP_K = min(20, len(bt_assets))
LOOKBACK_DAYS = 126

def build_weights_from_predictions(pred_matrix: pd.DataFrame, *, pm_style: str) -> pd.DataFrame:
    rebal_dates = set(pd.Series(pred_matrix.index, index=pred_matrix.index).resample(REBALANCE_FREQ).last().dropna().tolist())
    w_last = pd.Series(0.0, index=bt_assets)
    rows = []
    for dt in pred_matrix.index:
        if dt in rebal_dates:
            row = pred_matrix.loc[dt].dropna().sort_values(ascending=False)
            top = row.head(TOP_K)
            candidates = [a for a, v in top.items() if np.isfinite(v) and float(v) > 0.0]
            if not candidates:
                w_last = pd.Series(0.0, index=bt_assets)
            else:
                if pm_style == '1N':
                    w_dict = equal_weight(candidates)
                elif pm_style == 'MPT':
                    w_dict = optimize_mpt(returns_matrix, candidates, dt, lookback_days=LOOKBACK_DAYS)
                else:
                    raise ValueError(f'Unknown pm_style: {pm_style!r}')
                w_last = pd.Series(0.0, index=bt_assets)
                for a, w in w_dict.items():
                    w_last[str(a)] = float(w)
        rows.append(w_last)
    return pd.DataFrame(rows, index=pred_matrix.index, columns=bt_assets).fillna(0.0)

cfg = BacktestConfig(initial_equity=1_000_000.0, transaction_cost_bps=5.0, mode='vectorized')

compare_rows = []
results = {}
for pm_style in ['1N', 'MPT']:
    w = build_weights_from_predictions(pred_matrix, pm_style=pm_style)
    res = run_backtest(close_prices, w, config=cfg)
    rpt = compute_backtest_report(result=res, close_prices=close_prices)
    results[pm_style] = (w, res, rpt)
    compare_rows.append({
        'style': pm_style,
        'Total Return [%]': float(rpt['Total Return [%]']),
        'CAGR [%]': float(rpt['CAGR [%]']),
        'Sharpe': float(rpt['Sharpe']),
        'Max Drawdown [%]': float(rpt['Max Drawdown [%]']),
    })
compare = pd.DataFrame(compare_rows).sort_values('Total Return [%]', ascending=False).reset_index(drop=True)
display(compare)

BASE_TITLE = 'Stochastic Volatility (Particle Filter)'
for pm_style in ['1N', 'MPT']:
    w, res, rpt = results[pm_style]
    title = BASE_TITLE + ' - ' + pm_style
    display(rpt.to_frame(title))
    layout = build_interactive_portfolio_layout(
        market_ohlcv=market_df,
        equity=res.equity,
        returns=res.returns,
        weights=res.weights,
        turnover=res.turnover,
        costs=res.costs,
        close_prices=close_prices,
        title=title,
    )
    show(layout)


Loading BokehJS ...

,style,Total Return [%],CAGR [%],Sharpe,Max Drawdown [%]
0,1N,44.352866,27.727263,1.684742,-12.852631
1,MPT,31.682782,20.139548,1.222446,-15.593907


,Stochastic Volatility (Particle Filter) - 1N
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1443528.656117
Equity Peak,1464212.77093
Total Return [%],44.352866
CAGR [%],27.727263
Volatility (ann) [%],15.175843
Sharpe,1.684742


,Stochastic Volatility (Particle Filter) - MPT
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1316827.819201
Equity Peak,1324310.236462
Total Return [%],31.682782
CAGR [%],20.139548
Volatility (ann) [%],16.024232
Sharpe,1.222446
